In [136]:
import requests
import pandas as pd
import json as js
import sys
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.core.debugger import set_trace
from IPython.core.interactiveshell import InteractiveShell
import qgrid as qg
import unidecode
import datetime as dt
import numpy as np


InteractiveShell.ast_node_interactivity = "last"

In [137]:
# Pull League Info

league_id = 268439
year = 2022

url = (
    "https://fantasy.espn.com/apis/v3/games/fba/seasons/"
    + str(year)
    + "/segments/0/leagues/"
    + str(league_id)
)

# For Historical Seasons Use
# url = "https://fantasy.espn.com/apis/v3/games/fba/leagueHistory/" + \
#       str(league_id) + "?seasonId=" + str(year)

r = requests.get(url)

if r:
    print("Success!")
else:
    print("An error has occurred.")


leagueInfo = r.json()  ##View JSON structure

Success!


In [138]:
# Parse Team Data

teamData = [
    [
        team["id"],
        team["location"] + " " + team["nickname"],
        team["abbrev"],
        team["owners"],
    ]
    for team in leagueInfo["teams"]
]

teamData = pd.DataFrame(
    teamData, columns=["Team ID", "Team Name", "Abbreviation", "Owner(s)"]
)

teamData


,Team ID,Team Name,Abbreviation,Owner(s)
0,1,Steven Gerrard Slipped,23,"[{6F582A60-407B-4CF1-982A-60407B5CF1D0}, {51DC..."
1,2,Seattle BelowAvgSonics,SBAS,"[{B428156E-9B40-404B-A242-B9CED676B307}, {2A90..."
2,3,Foot Locker,😡🤣,"[{0CB5F7B3-8CD4-4CC9-B5F7-B38CD4CCC94B}, {4367..."
3,4,Orlando Magic,NANI,[{4A17167D-B662-4EDA-9A26-2ACF9F86A92D}]
4,5,Seattle NormalSonics,SEA,[{A75A7005-05F1-4DA3-B0EB-39B58DBD031B}]
5,6,Oklahoma City Thunder,OCYT,"[{D0EF20A3-9484-4D20-8DCC-8EE726B9C1FD}, {6605..."
6,8,"Mom, give me my PS4 back",PEEN,[{5FC573DF-D0A4-4F2D-BC29-368A7EB80C36}]
7,9,Susan .,sus,[{FD91D4CC-B9BA-47BC-9D3D-90D16201B4B7}]
8,10,Nikola Jokic,Dan,[{D53320EE-D95A-411D-B320-EED95AD11D18}]
9,11,Better Reich,WILL,[{09F9AF1E-0999-455B-B9AF-1E0999D55B81}]


In [139]:
# Request Roster information

r = requests.get(url, params={"view": "mRoster"})

if r:
    print("Success!")
else:
    print("An error has occurred.")

rosterInfo = r.json()

teamInfo = rosterInfo["teams"]  ##Team Data

Success!


In [140]:
# Parse Roster Data

leagueDF = pd.DataFrame()  # initialize league dataframe

# Dataframe Column Headers
cols = [
    "Fantasy Team ID",
    "Fantasy Team Name",
    "Abbrev.",
    "Player",
    "Fantasy Player ID",
    "Avg. Fantasy Points",
    "Total Fantasy Points",
    "Injury Status",
    "Acquisition Type",
]

for team in teamInfo:  # Parse each fantasy team
    teamDF = pd.DataFrame()  # initialize fantasy team dataframe
    teamID = team["id"]  # fantasy team id
    teamName = teamData[teamData["Team ID"] == teamID]["Team Name"].iloc[
        0
    ]  # associate fantasy team id with name
    teamAbbrev = teamData[teamData["Team ID"] == teamID]["Abbreviation"].iloc[
        0
    ]  # associate fantasy team id with abbrev.
    roster = team["roster"]

    # Parse every entry inside the roster
    for entry in roster["entries"]:
        acquisitionType = entry["acquisitionType"]

        # Parse all info inside the entry
        for playerEntry in entry["playerPoolEntry"]:
            playerID = entry["playerId"]
            entryObj = entry["playerPoolEntry"]

            # Parse player data within each entry
            if playerEntry == "player":
          
                for player in entryObj[playerEntry]:
                    statSourceID = 0
                    statSplitTypeID = 0
                    entryObj1 = entryObj[playerEntry]
                    injuryStatus = entryObj1["injuryStatus"]
                    #set_trace()
                    if player == "fullName":
                        playerName = entryObj1[player]
                    if player == "stats":

                        # Parse stats within the player data
                        for stats in entryObj1[player]:

                            # Parse each stat within stats
                            for stat in stats:
                                if stat == "statSourceId":
                                    statSourceID = stats[stat]
                                if stat == "statSplitTypeId":
                                    statSplitTypeID = stats[stat]
                                if stat == "seasonId":
                                    seasonId = stats[stat]

                                if stat == "appliedAverage":
                                        avgPoints = stats[stat]

                                if stat == "appliedTotal":
                                        totalPoints = stats[stat]

                                # Once we have the stats we need, make the dataframe
                                if (
                                    stat == "stats"
                                    and statSourceID == 0
                                    and statSplitTypeID == 3
                                    and seasonId == 2022
                                ):
                                    df = [
                                        teamID,
                                        teamName,
                                        teamAbbrev,
                                        playerName,
                                        playerID,
                                        avgPoints,
                                        totalPoints,
                                        injuryStatus,
                                        acquisitionType,
                                    ]
                                    df = pd.DataFrame(df).transpose()
                                    teamDF = teamDF.append(df, ignore_index=True)

    leagueDF = leagueDF.append(teamDF)

# Final League Dataframe
leagueDF.columns = cols
qg_widget = qg.show_grid(leagueDF, show_toolbar = True)
qg_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [141]:
# Define current Datetime 
currentTime = dt.datetime.now()

# Define function to find datetime nearest to current datetime
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


scheduleDF = pd.read_excel("LeagueSchedule.xlsx")

# Convert date headers to DateTime objects
weekDatesDF = scheduleDF.loc[:, scheduleDF.columns != "Tm"]
weekDatesDF.columns = pd.to_datetime(weekDatesDF.columns)

# Find the nearest fantasy week and create a dataframe with only that week's games
upcomingWeekDF = weekDatesDF.loc[:, nearest(weekDatesDF.columns, currentTime)]
upcomingWeekStr = "Week ~" + upcomingWeekDF.name.strftime("%D") + "~ # of Games"

fantasyWeek = nearest(weekDatesDF.columns, currentTime)

# Combine team and date dataframes
fullScheduleDF = pd.concat([scheduleDF["Tm"], weekDatesDF], axis=1)
fullScheduleDF

upcomingScheduleDF = pd.concat([fullScheduleDF["Tm"], upcomingWeekDF], axis=1)
upcomingScheduleDF = upcomingScheduleDF.rename(columns={fantasyWeek: upcomingWeekStr})
upcomingScheduleDF

,Tm,Week ~12/27/21~ # of Games
0,ATL,3
1,BKN,3
2,BOS,4
3,CHA,3
4,CHI,4
5,CLE,4
6,DAL,4
7,DEN,3
8,DET,2
9,GSW,3


In [41]:
# Player Stats for 2022
year = "2022"

url = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_per_game.html"

html = urlopen(url)

soup = BeautifulSoup(html)

# use findALL() to get the column headers
soup.findAll("tr", limit=1)
# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll("tr", limit=2)[0].findAll("th")]
# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
headers.insert(0, "Year")

# avoid the first header row
rows = soup.findAll("tr")[1:]
player_stats = [
    (td.getText() for td in rows[i].findAll("td")) for i in range(len(rows))
]

yearDF = pd.DataFrame([year] * len(rows))
playerStats = pd.DataFrame(player_stats)
playerStatsDF = pd.concat([yearDF, playerStats], axis=1)
playerStatsDF.columns = headers

# Clean player names
playerStatsDF["Player"] = playerStatsDF["Player"].apply(
    lambda x: unidecode.unidecode(str(x))
)

# qg_widget = qg.show_grid(playerStatsDF, show_toolbar = True)
# qg_widget

playerStatsDF

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2022,Precious Achiuwa,C,22,TOR,22,17,26.3,3.6,8.9,...,.528,2.2,5.9,8.1,1.5,0.5,0.5,1.1,2.5,8.4
1,2022,Steven Adams,C,28,MEM,33,33,25.2,2.5,4.9,...,.621,3.9,5.0,8.9,2.7,0.9,0.5,1.6,1.6,6.8
2,2022,Bam Adebayo,C,24,MIA,18,18,32.9,7.0,13.5,...,.759,2.7,7.4,10.2,3.2,1.1,0.3,2.9,3.3,18.7
3,2022,Santi Aldama,PF,21,MEM,16,0,9.8,1.5,4.1,...,.583,1.0,1.6,2.6,0.8,0.1,0.2,0.3,1.1,3.6
4,2022,LaMarcus Aldridge,C,36,BRK,25,8,23.6,6.0,10.4,...,.833,1.4,4.3,5.7,0.9,0.4,1.2,0.8,1.7,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,2022,Thaddeus Young,PF,33,SAS,22,0,14.1,2.9,5.1,...,.444,1.5,1.9,3.4,2.5,0.9,0.3,1.0,1.4,6.2
544,2022,Trae Young,PG,23,ATL,29,29,34.1,9.5,20.6,...,.891,0.8,3.2,4.0,9.3,1.0,0.1,4.1,1.7,27.3
545,2022,Omer Yurtseven,C,23,MIA,24,0,9.6,1.5,3.2,...,.654,1.1,2.7,3.8,0.5,0.2,0.5,0.8,1.2,3.7
546,2022,Cody Zeller,C,29,POR,24,0,13.3,1.8,3.2,...,.766,1.8,2.8,4.6,0.8,0.3,0.3,0.7,2.1,5.1


In [142]:
# Create df of games per player per fatnasy team of the upcoming week 
leagueStatsDF = pd.merge(leagueDF, playerStatsDF, on = "Player")


leagueScheduleDF = leagueStatsDF[
    [
        "Player",
        "Fantasy Player ID",
        "Tm",
        "Injury Status",
        "Age",
        "Pos",
        "Year",
        "Fantasy Team ID",
        "Fantasy Team Name",
        "Abbrev.",
        "Avg. Fantasy Points",
        "Total Fantasy Points",
        "Acquisition Type",
    ]
]

# leagueScheduleDF.loc[leagueScheduleDF["Fantasy Team ID"] > 5]
leagueScheduleWeekDF = pd.merge(leagueScheduleDF, upcomingScheduleDF, on="Tm")
leagueScheduleWeekDF.sort_values(by=["Fantasy Team Name"])
leagueScheduleWeekDF

,Player,Fantasy Player ID,Tm,Injury Status,Age,Pos,Year,Fantasy Team ID,Fantasy Team Name,Abbrev.,Avg. Fantasy Points,Total Fantasy Points,Acquisition Type,Week ~12/27/21~ # of Games
0,Luka Doncic,3945274,DAL,OUT,22,PG,2022,1,Steven Gerrard Slipped,23,39.1786,274.25,DRAFT,4
1,Kristaps Porzingis,3102531,DAL,ACTIVE,26,PF,2022,3,Foot Locker,😡🤣,30,330,DRAFT,4
2,Jalen Brunson,3934672,DAL,ACTIVE,25,PG,2022,4,Orlando Magic,NANI,25.3571,355,ADD,4
3,Tim Hardaway Jr.,2528210,DAL,OUT,29,SG,2022,11,Better Reich,WILL,21.625,259.5,DRAFT,4
4,Dorian Finney-Smith,2578185,DAL,ACTIVE,28,PF,2022,12,Seattle Supersonics,KHAT,22.6607,317.25,ADD,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Jaren Jackson Jr.,4277961,MEM,ACTIVE,22,PF,2022,9,Susan .,sus,25.8929,362.5,DRAFT,3
143,Dillon Brooks,3155526,MEM,OUT,26,SF,2022,10,Nikola Jokic,Dan,23.6429,331,DRAFT,3
144,De'Anthony Melton,4066436,MEM,OUT,23,SG,2022,11,Better Reich,WILL,22.1667,332.5,ADD,3
145,Ja Morant,4279888,MEM,ACTIVE,22,PG,2022,14,I'm now Third In Fantasy Prem,TEK,23.3333,70,DRAFT,3


In [143]:
# Game Limit Script- Upcoming Week 

# Sort by Injury Status
activePlayersDF = leagueScheduleWeekDF.loc[leagueScheduleWeekDF["Injury Status"] == "ACTIVE"]
dtdPlayersDF = leagueScheduleWeekDF.loc[leagueScheduleWeekDF["Injury Status"] == "DAY_TO_DAY"]
outPlayersDF = leagueScheduleWeekDF.loc[leagueScheduleWeekDF["Injury Status"] == "OUT"]

activePlayersGames = activePlayersDF[["Fantasy Team Name", upcomingWeekStr]].groupby(["Fantasy Team Name"]).sum()
activePlayersGames = activePlayersGames.rename(columns= {upcomingWeekStr: upcomingWeekStr + "~ ACTIVE"})
dtdPlayersGames = dtdPlayersDF[["Fantasy Team Name", upcomingWeekStr]].groupby(["Fantasy Team Name"]).sum()
dtdPlayersGames = dtdPlayersGames.rename(columns= {upcomingWeekStr: upcomingWeekStr + "~ Day-To-Day"})
outPlayersGames = outPlayersDF[["Fantasy Team Name", upcomingWeekStr]].groupby(["Fantasy Team Name"]).sum()
outPlayersGames = outPlayersGames.rename(columns= {upcomingWeekStr: upcomingWeekStr + "~ Out"})
totalPlayersGames = leagueScheduleWeekDF[["Fantasy Team Name", upcomingWeekStr]].groupby(["Fantasy Team Name"]).sum()
totalPlayersGames = totalPlayersGames.rename(columns= {upcomingWeekStr: upcomingWeekStr + "~ Total"})

totalPlayersGames = totalPlayersGames.merge(activePlayersGames, on = "Fantasy Team Name", how = "left")
totalPlayersGames = totalPlayersGames.merge(dtdPlayersGames, on = "Fantasy Team Name", how = "left")
totalPlayersGames = totalPlayersGames.merge(outPlayersGames, on = "Fantasy Team Name", how = "left")
totalPlayersGames 



,Week ~12/27/21~ # of Games~ Total,Week ~12/27/21~ # of Games~ ACTIVE,Week ~12/27/21~ # of Games~ Day-To-Day,Week ~12/27/21~ # of Games~ Out
Fantasy Team Name,,,,
Better Reich,44,25,NaN,19
Foot Locker,45,32,3.0,10
I'm now Third In Fantasy Prem,39,31,NaN,8
"Mom, give me my PS4 back",35,31,NaN,4
Nikola Jokic,45,24,NaN,21
Oklahoma City Thunder,38,18,7.0,13
Orlando Magic,50,43,NaN,7
Seattle BelowAvgSonics,41,22,NaN,19
Seattle NormalSonics,53,38,NaN,15


In [144]:
# Game Limit Script- Upcoming Week 

totalPlayersGames.describe()

,Week ~12/27/21~ # of Games~ Total,Week ~12/27/21~ # of Games~ ACTIVE,Week ~12/27/21~ # of Games~ Day-To-Day,Week ~12/27/21~ # of Games~ Out
count,12.000000,12.000000,2.000000,12.000000
mean,43.166667,29.750000,5.000000,12.583333
std,5.507571,6.903556,2.828427,6.141636
min,35.000000,18.000000,3.000000,3.000000
25%,38.750000,24.750000,4.000000,7.750000
50%,44.500000,31.000000,5.000000,13.500000
75%,46.000000,32.250000,6.000000,18.250000
max,53.000000,43.000000,7.000000,21.000000


In [135]:
# Game Limit Script -  Season

m = pd.concat(
    [leagueScheduleFullDF["Fantasy Team Name"], leagueScheduleFullDF[weekDatesDF.columns]],
    axis=1,
)
# np.floor(m.groupby(["Fantasy Team Name"]).mean().mean()*13-5)
m.groupby(["Fantasy Team Name"]).sum().describe()

,2021-10-18,2021-10-25,2021-11-01,2021-11-08,2021-11-15,2021-11-22,2021-11-29,2021-12-06,2021-12-13,2021-12-20,...,2022-01-31,2022-02-07,2022-02-14,2022-02-21,2022-02-28,2022-03-07,2022-03-14,2022-03-21,2022-03-28,2022-04-04
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,...,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,32.333333,42.916667,43.916667,41.500000,44.166667,42.583333,39.000000,41.750000,43.083333,36.166667,...,41.083333,38.750000,25.000000,24.916667,39.500000,42.833333,42.333333,40.666667,46.500000,42.750000
std,2.806918,5.367551,5.418123,4.889692,6.379418,5.212892,5.510321,4.474676,5.350588,4.041452,...,6.556861,4.864061,3.247377,3.396745,4.776838,5.024184,4.886593,5.262791,5.452272,4.351071
min,27.000000,34.000000,33.000000,34.000000,30.000000,35.000000,28.000000,34.000000,34.000000,28.000000,...,31.000000,30.000000,18.000000,20.000000,29.000000,34.000000,37.000000,30.000000,37.000000,37.000000
25%,30.500000,39.500000,41.000000,37.500000,42.250000,39.250000,35.500000,38.750000,38.000000,35.250000,...,36.250000,36.750000,24.750000,22.750000,38.000000,39.750000,37.750000,38.500000,45.000000,38.750000
50%,33.000000,43.000000,45.000000,42.000000,45.500000,42.500000,40.500000,42.000000,45.000000,37.000000,...,42.500000,39.000000,26.000000,25.000000,40.000000,42.500000,41.000000,41.500000,47.500000,43.500000
75%,34.250000,48.000000,47.250000,46.250000,47.500000,45.250000,43.000000,43.500000,47.000000,38.250000,...,45.250000,41.750000,27.000000,27.250000,42.000000,45.750000,46.250000,43.750000,50.000000,45.250000
max,36.000000,50.000000,52.000000,47.000000,52.000000,54.000000,47.000000,51.000000,50.000000,42.000000,...,52.000000,46.000000,28.000000,30.000000,47.000000,50.000000,51.000000,48.000000,55.000000,51.000000
